In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [2]:
using LinearAlgebra, Combinatorics, Printf, Dates, PythonCall

sympy = pyimport("sympy")

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")
include("DefineSymbols.jl")
include("SolveVars.jl")
include("DefineAction.jl")
include("ComputeEOMs.jl")

using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU
using .CriticalPoints: compute_bdy_critical_data
using .DefineSymbols: run_define_variables
using .SolveVars: run_solver
using .DefineAction: vertexaction
using .DefineAction: run_action
using .EOMs: compute_EOMs, zero_or_not

In [3]:
bdypoints_all = [[[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0],
 [0.0, 1.0, 1.0, 0.0],
 [0.5, 1.0, 1.0, 0.0],
 [0.5, 1.0, 1.0, 1.0]]]

1-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 1.0, 0.0], [0.5, 1.0, 1.0, 0.0], [0.5, 1.0, 1.0, 1.0]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    bdypoints = bdypoints_all[s]
    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:


In [6]:
bdypoints = bdypoints_all[1]

5-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0]
 [0.0, 1.0, 1.0, 0.0]
 [0.5, 1.0, 1.0, 0.0]
 [0.5, 1.0, 1.0, 1.0]

In [19]:
1/2

0.5

In [5]:
geom = GeometryCollection(datasets);

In [6]:
four_simplices = [[1,2,3,4,5], [1,2,3,5,6], [1,3,4,5,6]];

fix_kappa_signs!(four_simplices, geom);

build_global_connectivity(four_simplices, geom);

[1, 2, 3]


In [7]:
conn = build_global_connectivity(four_simplices, geom)
push!(geom.connectivity, conn);

In [8]:
run_face_xi_matching(geom);

  SL(2,C) matrices updated:      ✓
  Boundary bivectors updated:    ✓
  boundary ξ variables updated:  ✓
  SU(2)/SU(1,1) elements updated: ✓
  SO(1,3) frames corrected:       ✓



In [9]:
check_all(geom)


   FACE–XI GEOMETRY CHECKS

✓ Face matching satisfied (max residual = 2.3339921916043816e-16).
✓ SL(2,C) parallel transport satisfied (max residual = 1.2075270435462621e-15).
✓ Closure condition satisfied (max residual = 1.6386370948030818e-15).

------------------------------
✓ All geometric consistency checks PASSED.
------------------------------



In [10]:
run_su2_su11_gauge_fix(geom);

In [11]:
run_define_variables(geom);

In [12]:
sol_vars, sol_bdry, γsym = run_solver(geom);

In [13]:
length(sol_vars)

124

In [14]:
S = run_action(geom);

In [16]:
S_vars = S.xreplace(sol_bdry);

In [17]:
expr = S_vars.subs(sol_vars);

In [18]:
S_simp = sympy.simplify(sympy.simplify(expr).evalf())

Python: -8.03262776367368e-16 + 0.219402265918146*I + 7.19424519957101e-14/gamma - 12.3764773510103*I/gamma

In [93]:
using PythonCall
sympy = pyimport("sympy")
γsym = sympy.symbols("gamma", real=true)

function compute_EOMs_new(S::Py, sol_vars::Dict{Py,Py}; γ::Union{Py,Nothing}=γsym,γval=1)

    dS_eval = Dict{Py,Py}()

    for v in keys(sol_vars)
        # 1. symbolic derivative
        expr = sympy.diff(S, v)

        # 2. substitute solution
        expr = expr.subs(sol_vars)

        # 3. substitute gamma if requested
         if γ !== nothing
            expr = expr.subs(γ, γval)
        end

        # 4. numeric evaluation
        expr = expr.evalf(chop=true)

        dS_eval[v] = expr
    end

    return dS_eval
end

compute_EOMs_new (generic function with 1 method)

In [94]:
results = compute_EOMs_new(S_vars, sol_vars; γ = γsym);

In [98]:
using PythonCall

function zero_or_not(dS_eval; tol = 1e-10)

    all_zero = true

    for (v, val) in dS_eval
        # extract real / imaginary parts via SymPy
        re_part = sympy.re(val).evalf()
        im_part = sympy.im(val).evalf()

        # convert Python → Julia safely
        re_val = abs(pyconvert(Float64, re_part))
        im_val = abs(pyconvert(Float64, im_part))

        if re_val > tol || im_val > tol
            println("dS/d$(v) = $(val)  ≠ 0  ✘")
            all_zero = false
        end
    end

    if all_zero
        println("✔ All equations of motion are satisfied (within tol = $tol).")
    end

    return nothing
end

zero_or_not (generic function with 1 method)

In [99]:
zero_or_not(results; tol = 1e-10)

✔ All equations of motion are satisfied (within tol = 1.0e-10).
